# Chap03 - 딕셔너리와 집합

> 프로그램 코드 안에서 명시적으로 딕셔너리 사용하고 있지 않더라도, 모든 파이썬 프로그램에서는 여러 딕셔너리가 동시에 활동하고 있다.
-A.M 커츨링-

파이썬의 딕셔너리(dictionary)와 집합(set)은 [**해시 테이블**](https://ko.wikipedia.org/wiki/%ED%95%B4%EC%8B%9C_%ED%85%8C%EC%9D%B4%EB%B8%94)을 이용해 구현되었다. 

이번 Chap03 에서 다룰 내용은 다음과 같다.

 - Common dictionary methods
 - Special handling for missing keys
 - Variations of dict in the standard library
 - The set and frozenset types
 - How hash tables work
 - Implications of hash tables: key type limitations, unpredictable ordering etc.

## 3.1 Generic Mapping Types

`collections.abc` 모듈은 `dict`(딕셔너리) 및 유사한 자료형의 인터페이스를 정의하는 `Mapping` 및 `MutalbelMapping` 추상 클래스(ABC)를 제공한다.

![](./images/3-1.PNG)

아래의 예제에서도 확인할 수 있듯이, `isinstance()` 함수를 이용해 딕셔너리(`dict`)는 `collections.abc.Mapping`의 자료형임을 알 수 있다.

In [1]:
import collections

my_dict = {}
isinstance(my_dict, collections.abc.Mapping)

True

파이썬에서 제공하는 매핑형(mapping type)은 모두 `dict`를 이용해 구현되기 때문에, 키(key)가 **해시 가능([hashable](https://docs.python.org/3/glossary.html#term-hashable))**해야 한다.

> hashable이란 변하지 않는 해시값을 가지고 있고 다른 객체와 비교할 수 있으면, 이 객체를 hashable이라고 한다.

위의 설명에서도 유추할 수 있듯이 불변형인 `str, byte, tuple, ...`은 hashable 하다. 

아래의 예제코드에서 `hash()`함수는 해당 객체의 해시값을 반환해준다.

In [2]:
tt = (1, 2, (30, 40))
hash(tt)

8027212646858338501

아래의 예제코드에서 에러가 나는 이유는 튜플(tuple)안에 가변 시퀀스인 리스트(`list`)가 들어있기 때문이다.

In [3]:
t1 = (1, 2, [30, 40])
hash(t1)

TypeError: unhashable type: 'list'

마찬가지로 `set`(집합) 또한 가변 시퀀스이기 때문에, `hash()`함수를 적용하면 에러가 난다.

In [6]:
tf = (1, 2, set([30, 40]))
hash(tf)

TypeError: unhashable type: 'set'

`set`과 같은 기능을 하는 [`frozenset`](https://docs.python.org/ko/3.6/library/stdtypes.html?highlight=frozenset#frozenset)은 hashable 하다.

In [7]:
tf2 = (1, 2, frozenset([30, 40]))
hash(tf2)

985328935373711578

파이썬 [공식문서](https://docs.python.org/ko/3/library/stdtypes.html#mapping-types-dict)에서도 확인할 수 있듯이, 아래와 같이 다양한 방식으로 딕셔너리를 구현할 수 있다.

In [14]:
a = dict(one=1, two=2, three=3)
b = {'one': 1, 'two': 2, 'three': 3}
c = dict(zip(['one', 'two', 'three'], [1, 2, 3]))
d = dict([('two', 2), ('one', 1), ('three', 3)])
e = dict({'three': 3, 'one': 1, 'two': 2})

print(a == b == c == d == e)
print(a)

True
{'one': 1, 'two': 2, 'three': 3}


## 3.2 딕셔너리 컴프리헨션

[2장-시퀀스](http://excelsior-cjh.tistory.com/164)에서 살펴본 리스트 컴프리헨션과 동일하게 딕셔너리 또한 **딕셔너리 컴프리헨션(dict comprehension)**을 이용해 딕셔너리 객체를 만들 수 있다.

In [17]:
DIAL_CODES = [(82, 'Korea'),
              (86, 'China'),
              (91, 'India'),
              (1, 'United States'),
              (62, 'Indonesia'),
              (55, 'Brazil'),
              (92, 'Pakistan'),
              (880, 'Bangladesh'),
              (234, 'Nigeria'),
              (7, 'Russia')]

country_code = {country.upper(): code for code, country in DIAL_CODES}
print(country_code)

{'KOREA': 82, 'CHINA': 86, 'INDIA': 91, 'UNITED STATES': 1, 'INDONESIA': 62, 'BRAZIL': 55, 'PAKISTAN': 92, 'BANGLADESH': 880, 'NIGERIA': 234, 'RUSSIA': 7}


## 3.3 공통적인 매핑 메서드